In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Double check timestamp of sentences

## Import module and utilities

In [3]:
import json
import pandas as pd
import pprint
import matplotlib.pyplot as plt
import numpy as np
# !pip install -Iv seaborn==0.11.0
import seaborn as sns
print(sns.__version__)
sns.set_style()
from scipy.stats import pearsonr
from scipy.stats import iqr

%cd '/content/drive/My Drive/Colab Notebooks/media-agenda'
%pwd 

!pip install arch
import senti_util
import util
from util import DocType, Polarity, Source, OptimalKClustersConfig

from termcolor import colored, cprint

0.11.0
/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda
     |████████████████████████████████| 798kB 5.4MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Issue 1: Both article and comment posting time as aligned as the article posting time

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/sentence_cluster_sentiment_dict.json'

with open(path, 'r') as f:
  assignments = json.load(f)

print('Loaded total {} cluster-sentiment assignments'.format(len(assignments)))
print('Example of an assignment object')
pprint.pprint(assignments[0])

# Convert assignments into panda dataframe
df = pd.DataFrame.from_dict(assignments)
df['date_str'] = df['date']
df['month'] = df['date'].str[:-3]
df['year'] = df['date'].str[:4]
df['date'] = pd.to_datetime(df['date'])
df['is_comment'] = df['com_id'] != DocType.NON_COMMENT.value

print()

# Filter out unnecessary the assignment records
# 1. Records from garbage clusters
# 2. Records not falls between 2009 and 2017
# pprint.pprint(pd.unique(df['cluster']))
df = df[~df['cluster'].isin(OptimalKClustersConfig.garbage_clusters)]
print('Unique cluster after filtering:')
pprint.pprint(pd.unique(df['cluster']))

start_year = 2009
end_year = 2017
start_datetime, end_datetime, start_datetime_str, end_datetime_str = SentiUtil.get_start_end_datetime(start_year, end_year)

df = df[(df['date'] >= start_datetime) & (df['date'] <= end_datetime)]
df = df.reset_index()
print('Min year after filtering:', min(df['year']))
print('Max yearafter filtering:', max(df['year']))
print('Number of sentences after filtering:', df.shape[0])
display(pd.DataFrame(columns = ['Source', 'Total sentences', 'Article sentences', 'Comment sentences'],
             data = [[Source.NYTIMES, df[df.corpus_name == Source.NYTIMES].shape[0], 
                      df[(df.corpus_name == Source.NYTIMES) & (df.is_comment == False)].shape[0],
                      df[(df.corpus_name == Source.NYTIMES) & (df.is_comment == True)].shape[0]], 
                     [Source.QUORA, df[df.corpus_name == Source.QUORA].shape[0], 
                      df[(df.corpus_name == Source.QUORA) & (df.is_comment == False)].shape[0],
                      df[(df.corpus_name == Source.QUORA) & (df.is_comment == True)].shape[0]],
                     [Source.SPIEGEL, df[df.corpus_name == Source.SPIEGEL].shape[0], 
                      df[(df.corpus_name == Source.SPIEGEL) & (df.is_comment == False)].shape[0],
                      df[(df.corpus_name == Source.SPIEGEL) & (df.is_comment == True)].shape[0]],]))
print()
print('Display top 5 row of the preprocessed dataframe:')
display(df.head())

Loaded total 328091 cluster-sentiment assignments
Example of an assignment object
{'cluster': 1,
 'com_id': -1,
 'corpus_name': 'nytimes',
 'date': '2005-11-01',
 'doc_id': 0,
 'global_id': 0,
 'sentiment': 0.125}

Unique cluster after filtering:
array([ 1,  3, 10, 12,  6, 13,  8, 11,  0,  5,  7])


NameError: ignored

In [ ]:
print(df[(df.corpus_name == 'nytimes') & (df.doc_id == 310) & (df.is_comment == True)].date.unique())
df[(df.corpus_name == 'nytimes') & (df.doc_id == 310) & (df.is_comment == False)].date.unique()

In [ ]:
# check if the article posting date is unique for every article
article_posting_date = df[(df.corpus_name == Source.NYTIMES) & (df.is_comment == False)].groupby('doc_id').date.unique()
for index, value in article_posting_date.items():
  if len(value) != 1:
    print(f"Index : {index}, Value : {value}")
print('Checking completed!')

Checking completed!


In [ ]:
# check if the comment posting date is unique for every article
comment_posting_date = df[(df.corpus_name == Source.NYTIMES) & (df.is_comment == True)].groupby('doc_id').date.unique()
for index, value in comment_posting_date.items():
  if len(value) != 1:
    print(f"Index : {index}, Value : {value}")
print('Checking completed!')

Checking completed!


In [ ]:
# check if the comment posting date is exactly the same as that of article for every articles
doc_id_posting_date = df[(df.corpus_name == Source.NYTIMES)].groupby('doc_id').date.unique()
for index, value in doc_id_posting_date.items():
  if len(value) != 1:
    print(f"Index : {index}, Value : {value}")
print('Checking completed!')

grouped_df = df[(df.corpus_name == Source.NYTIMES)].groupby('doc_id').date
for key, item in grouped_df:
  print(grouped_df.get_group(key), "\n\n")
  if key >= 5:
    break

Checking completed!
0   2016-04-25
1   2016-04-25
2   2016-04-25
3   2016-04-25
4   2016-04-25
5   2016-04-25
6   2016-04-25
Name: date, dtype: datetime64[ns] 


7    2012-09-08
8    2012-09-08
9    2012-09-08
10   2012-09-08
11   2012-09-08
12   2012-09-08
13   2012-09-08
14   2012-09-08
15   2012-09-08
16   2012-09-08
17   2012-09-08
18   2012-09-08
19   2012-09-08
20   2012-09-08
21   2012-09-08
22   2012-09-08
23   2012-09-08
24   2012-09-08
25   2012-09-08
26   2012-09-08
27   2012-09-08
28   2012-09-08
29   2012-09-08
30   2012-09-08
31   2012-09-08
32   2012-09-08
33   2012-09-08
34   2012-09-08
35   2012-09-08
36   2012-09-08
37   2012-09-08
38   2012-09-08
39   2012-09-08
40   2012-09-08
41   2012-09-08
42   2012-09-08
43   2012-09-08
44   2012-09-08
45   2012-09-08
46   2012-09-08
47   2012-09-08
48   2012-09-08
49   2012-09-08
50   2012-09-08
51   2012-09-08
52   2012-09-08
Name: date, dtype: datetime64[ns] 


53   2016-09-02
54   2016-09-02
55   2016-09-02
56   2016-09-02
5

## Solution for issue 1: Assign article time for article content and comment time for corresponding comment

Detail see regenerate_sentences_metadata_json.ipynb

## Issue 2: comment posting time earlier than article time

Solution: assign the article date with the earliest comment date

In [4]:
nytimes = pd.read_json('/content/drive/My Drive/Colab Notebooks/media-agenda/data/media-agenda-dataset-quora/nytimes.json')
spiegel = pd.read_json('/content/drive/My Drive/Colab Notebooks/media-agenda/data/media-agenda-dataset-quora/spiegel.json')
quora = pd.read_json('/content/drive/My Drive/Colab Notebooks/media-agenda/data/media-agenda-dataset-quora/quora.json')

nytimes = nytimes[nytimes['relevant'] == 1.0]
spiegel = spiegel[spiegel['relevant'] == 1.0]
quora = quora[quora['relevant'] == 1.0]

In [ ]:
from datetime import datetime 

article_posting_time = nytimes[nytimes['comments'].str.len() != 0].article_time
for index, value in nytimes[nytimes['comments'].str.len() != 0].comments.items():

  # print(article_posting_time[index])
  for i in range(len(value)):
    if datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S') <= article_posting_time[index]:
      if datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S').date() < article_posting_time[index].date():
        print(index, value[i]['comment_time'], article_posting_time[index]) 
print('completed!')

nytimes.loc[419] == nytimes.iloc[310] # https://www.nytimes.com/roomfordebate/2012/09/10/is-organic-food-worth-the-expense/buying-organic-fruits-and-vegetables-is-a-personal-choice
# updated the article time from its updated time 2014-02-06 00:00:00 to 2012-09-10 00:00:00??

In [ ]:
from datetime import datetime 
article_text = quora[quora['comments'].str.len() != 0].article_text
article_posting_time = quora[quora['comments'].str.len() != 0].article_time
for index, value in quora[quora['comments'].str.len() != 0].comments.items():
  # print(article_posting_time[index])
  for i in range(len(value)):
    if datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S') <= article_posting_time[index]:
      if datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S').date() < article_posting_time[index].date(): # 7 cases
        if article_text[index] != '': # 3 cases
          print(index, value[i]['comment_time'], article_posting_time[index]) 

print('completed!')

# quora.loc[397].comments
# remove the comment posted on 2013-08-01 00:00:00??
# quora.loc[853].comments
# remove the comment posted on 2015-08-25 00:00:00 and 2016-01-05 00:00:00??
# quora.loc[988] # https://www.quora.com/Why-is-organic-food-more-expensive-than-other-food
# remove the comment posted on 2012-06-13 00:00:00??

397 2013-08-01 00:00:00 2014-06-09 00:16:00
853 2015-08-25 00:00:00 2016-01-18 17:54:00
853 2016-01-05 00:00:00 2016-01-18 17:54:00
988 2012-06-13 00:00:00 2015-08-21 04:18:00
completed!


In [ ]:
from datetime import datetime 
article_text = spiegel[spiegel['comments'].str.len() != 0].article_text
article_posting_time = spiegel[spiegel['comments'].str.len() != 0].article_time
indeces = []
count = {}
total = {}
url = {}
min_date = {}
max_date = {}
for index, value in spiegel[spiegel['comments'].str.len() != 0].comments.items():
  # print(article_posting_time[index])
  for i in range(len(value)):
    if index in (135, 145, 160, 166, 173, 177, 191, 196, 213, 220, 273, 283):
      total[index] = total[index] + 1 if index in total else 1
    if datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S') <= datetime.strptime(article_posting_time[index], '%Y-%m-%d %H:%M:%S'):
      if datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S').date() < datetime.strptime(article_posting_time[index], '%Y-%m-%d %H:%M:%S').date():
        indeces.append(index)
        if index in (135, 145, 160, 166, 173, 177, 191, 196, 213, 220, 273, 283):
          count[index] = count[index] + 1 if index in count else 1
          comment_datetime = datetime.strptime(value[i]['comment_time'], '%Y-%m-%d %H:%M:%S')
          min_date[index] = comment_datetime if (index not in min_date or min_date[index] > comment_datetime) else min_date[index]
          max_date[index] = comment_datetime if (index not in max_date or max_date[index] < comment_datetime) else max_date[index]
        # if index == 135:
        #   # print(index, value[i]['comment_time'], article_posting_time[index]) 
        #   count = count + 1
# print('count:', count, 'out of', total)
# print('completed!')
# display(set(indeces)) # {135, 145, 160, 166, 173, 177, 191, 196, 213, 220, 273, 283}

article_time = []
url = []
for i in sorted(set(indeces)):
  article_time.append(spiegel.loc[i].article_time)
  url.append(spiegel.loc[i].article_url)
  
spiegel.loc[{135, 145, 160, 166, 173, 177, 191, 196, 213, 220, 273, 283}].article_time
# spiegel.loc[135].comments
# have 1 out of 408 incorrected dated comments, remove the comment posted on 2008-08-19 15:59:00??
# spiegel.loc[145].comments[-1]
# have 3348 out of 3812 incorrected dated comments, with the earliest comment posting date as 2009-07-10 23:28:00 while article lasted modified date as 2009-07-31 16:01:55
# spiegel.loc[160].comments
# have 137 out of 165 incorrected dated comments, with the comment posting date ranging 2007-08-28 12:59:00 to 2011-02-27 13:15:00 while article lasted modified date as 2007-12-22 08:22:24

result_df = pd.DataFrame({'index': sorted(set(indeces)), 
                          'num_of_incorrect_dated_comments': count.values(), 
                          'total_number_of_comments': total.values(),
                          'article_time': article_time,
                          'min_incorrect_date': min_date.values(),
                          'max_incorrect_date': max_date.values(),
                          'url': url})

result_df

,index,num_of_incorrect_dated_comments,total_number_of_comments,article_time,min_incorrect_date,max_incorrect_date,url
0,135,1,408,2008-08-20 11:24:52,2008-08-19 15:59:00,2008-08-19 15:59:00,http://www.spiegel.de/wirtschaft/biosupermarkt...
1,145,3348,3812,2009-07-31 16:01:55,2009-07-10 00:51:00,2009-07-30 17:43:00,http://www.spiegel.de/wirtschaft/imitate-essen...
2,160,137,165,2007-12-22 08:22:24,2007-08-28 12:59:00,2007-10-16 07:54:00,http://www.spiegel.de/wirtschaft/verbrauchersc...
3,166,23,206,2007-06-12 18:54:41,2007-06-11 12:52:00,2007-06-11 22:33:00,http://www.spiegel.de/wirtschaft/oeko-produkte...
4,173,58,98,2009-02-21 12:36:23,2008-11-13 22:49:00,2008-11-26 13:06:00,http://www.spiegel.de/wirtschaft/biofach-messe...
5,177,41,98,2008-11-25 06:04:32,2008-11-13 22:49:00,2008-11-20 00:21:00,http://www.spiegel.de/wirtschaft/schokoladenko...
6,191,282,408,2009-03-07 11:54:55,2008-08-19 15:59:00,2008-09-13 23:26:00,http://www.spiegel.de/wirtschaft/bio-lebensmit...
7,196,229,408,2008-08-31 18:58:58,2008-08-19 15:59:00,2008-08-30 22:10:00,http://www.spiegel.de/wirtschaft/bio-festival-...
8,213,1222,2772,2011-01-07 19:44:00,2011-01-04 11:12:00,2011-01-06 23:44:00,http://www.spiegel.de/wirtschaft/unternehmen/d...
9,220,1,98,2008-11-14 13:07:01,2008-11-13 22:49:00,2008-11-13 22:49:00,http://www.spiegel.de/wirtschaft/oeko-boom-bio...


In [ ]:
print(nytimes.shape[0], quora.shape[0], spiegel.shape[0])

327 1497 152


## Solution for issue 2: Reassign article time as earliest commment time

* The following function is adopted in senti_util.py

In [ ]:
def get_sentence_cluster_sentiment_df(start_year = 2009, end_year = 2017, path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json', verbose = True):

  with open(path, 'r') as f:
    assignments = json.load(f)

  if verbose:
    print('Loaded total {} cluster-sentiment assignments'.format(len(assignments)))
    print('Example of an assignment object')
    print()

  # Convert assignments into panda dataframe
  df = pd.DataFrame.from_dict(assignments)
  df['is_comment'] = df['comment_id'] != DocType.NON_COMMENT.value
  df['posting_time'] = pd.to_datetime(df['posting_time'])

  if verbose:
    print('Number of article from:')
    display(pd.DataFrame([('nytimes', len(df[df.source == 'nytimes'].article_id.unique())),
                          ('quora', len(df[df.source == 'quora'].article_id.unique())),
                          ('spiegel', len(df[df.source == 'spiegel'].article_id.unique())),
                          ('total', len(df[df.source == 'nytimes'].article_id.unique()) + len(df[df.source == 'quora'].article_id.unique()) + len(df[df.source == 'spiegel'].article_id.unique()))],
                        columns = ['source', 'num_of_article']))
    print()

  # Update the article posting time as its earliest comment posting time if there is any comment posting is earlier than the original article time
  article_posting_time_df = df[df.is_comment == False].groupby(['source', 'article_id']).posting_time.min().reset_index()
  comment_earliest_posting_time_df = df[df.is_comment == True].groupby(['source', 'article_id']).posting_time.min().reset_index()
  merged_df = pd.merge(article_posting_time_df, comment_earliest_posting_time_df, how = 'outer', on = ['source', 'article_id'], validate = 'one_to_one')

  incorrect_dated_article = []
  for index, row in merged_df.iterrows():
    if row['posting_time_x'].date() > row['posting_time_y'].date():
      incorrect_dated_article.append((row['source'], row['article_id'], row['posting_time_x'], row['posting_time_y'].date()))
      df.loc[(df.is_comment == False) & (df.source == row['source']) & (df.article_id == row['article_id']), 'posting_time'] = row['posting_time_y'].date()

  # create new columns
  df['posting_time'] = pd.to_datetime(df['posting_time'])
  df['date'] = df['posting_time'].dt.date
  df['month'] = df['posting_time'].dt.month
  df['year'] = df['posting_time'].dt.year

  # Filter out unnecessary the assignment records
  # 1. Records from garbage clusters
  # 2. Records not falls between 2009 and 2017
  # pprint.pprint(pd.unique(df['cluster']))
  df = df[~df['cluster'].isin(OptimalKClustersConfig.garbage_clusters)]

  if verbose:
    print('Unique cluster after filtering:')
    pprint.pprint(pd.unique(df['cluster']))
    print()

  start_datetime, end_datetime, start_datetime_str, end_datetime_str = util.get_start_end_datetime(start_year, end_year)

  df = df[(df['date'] >= start_datetime) & (df['date'] <= end_datetime)]
  df = df.reset_index()
  if verbose:
    print('Min year after filtering:', min(df['year']))
    print('Max yearafter filtering:', max(df['year']))
    print('Number of sentences after filtering:', df.shape[0])
    display(pd.DataFrame(columns = ['Source', 'Total sentences', 'Article sentences', 'Comment sentences'],
                data = [[Source.NYTIMES, df[df.source == Source.NYTIMES].shape[0], 
                          df[(df.source == Source.NYTIMES) & (df.is_comment == False)].shape[0],
                          df[(df.source == Source.NYTIMES) & (df.is_comment == True)].shape[0]], 
                        [Source.QUORA, df[df.source == Source.QUORA].shape[0], 
                          df[(df.source == Source.QUORA) & (df.is_comment == False)].shape[0],
                          df[(df.source == Source.QUORA) & (df.is_comment == True)].shape[0]],
                        [Source.SPIEGEL, df[df.source == Source.SPIEGEL].shape[0], 
                          df[(df.source == Source.SPIEGEL) & (df.is_comment == False)].shape[0],
                          df[(df.source == Source.SPIEGEL) & (df.is_comment == True)].shape[0]],]))
    print()
    print('Display top 5 row of the preprocessed dataframe:')
    display(df.head())
    print()

  if verbose:
    print('Reference:')
    print('Table showing the articles with posting time updated ')
    display(pd.DataFrame(incorrect_dated_article, columns = ['source', 'article_id', 'changed from', 'to']))
    print()
    print('Current values for the 12th article above:')
    print('Source:', incorrect_dated_article[12][0], ' Article_id:', incorrect_dated_article[12][1])
    display(df[(df.is_comment == False) & (df.source == incorrect_dated_article[12][0]) & (df.article_id == incorrect_dated_article[12][1])])

  return df

get_sentence_cluster_sentiment_df(start_year = 1900, end_year = 2020)

Loaded total 328091 cluster-sentiment assignments
Example of an assignment object

Number of article from:


,source,num_of_article
0,nytimes,327
1,quora,1338
2,spiegel,152
3,total,1817



Unique cluster after filtering:
array([ 1,  3, 10, 12,  6, 13,  8, 11,  0,  5,  7])

Min year after filtering: 1970
Max yearafter filtering: 2017
Number of sentences after filtering: 232103


,Source,Total sentences,Article sentences,Comment sentences
0,nytimes,59799,14202,45597
1,quora,44140,1022,43118
2,spiegel,128164,6795,121369



Display top 5 row of the preprocessed dataframe:


...


Reference:
Table showing the articles with posting time updated 


,source,article_id,changed from,to
0,nytimes,310,2014-02-06 00:00:00,2012-09-11
1,quora,331,2014-06-09 00:16:00,2013-08-01
2,quora,736,2016-01-18 17:54:00,2015-08-25
3,quora,852,2015-08-21 04:18:00,2012-06-13
4,spiegel,39,2008-08-20 11:24:52,2008-08-19
5,spiegel,46,2009-07-31 16:01:55,2009-07-10
6,spiegel,54,2007-12-22 08:22:24,2007-08-28
7,spiegel,56,2007-06-12 18:54:41,2007-06-11
8,spiegel,59,2009-02-21 12:36:23,2008-11-13
9,spiegel,62,2008-11-25 06:04:32,2008-11-13



Current values for the 12th article above:
Source: spiegel  Article_id: 78


,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
164835,222919,239782,spiegel,78,-1,2011-01-04,3,0.0000,0.000000,0.000000,False,2011-01-04,1,2011
164836,222920,239783,spiegel,78,-1,2011-01-04,13,0.0000,0.000000,1.000000,False,2011-01-04,1,2011
164837,222921,239784,spiegel,78,-1,2011-01-04,11,0.0040,0.000000,0.000000,False,2011-01-04,1,2011
164838,222922,239785,spiegel,78,-1,2011-01-04,13,-0.6621,-0.700000,0.000000,False,2011-01-04,1,2011
164839,222923,239786,spiegel,78,-1,2011-01-04,12,-0.3233,-0.666667,0.333333,False,2011-01-04,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164897,222985,239848,spiegel,78,-1,2011-01-04,10,0.0000,0.000000,0.000000,False,2011-01-04,1,2011
164898,222986,239849,spiegel,78,-1,2011-01-04,3,0.0000,0.000000,0.000000,False,2011-01-04,1,2011
164899,222987,239850,spiegel,78,-1,2011-01-04,12,-0.0048,-0.700000,0.000000,False,2011-01-04,1,2011
164900,222988,239851,spiegel,78,-1,2011-01-04,12,0.0000,0.000000,0.000000,False,2011-01-04,1,2011


,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
0,0,0,nytimes,0,-1,2005-11-01 00:00:00,1,0.12500,-0.017045,0.518182,False,2005-11-01,11,2005
1,1,1,nytimes,0,-1,2005-11-01 00:00:00,1,-0.12500,0.337500,0.675000,False,2005-11-01,11,2005
2,2,2,nytimes,0,-1,2005-11-01 00:00:00,3,0.07500,0.085714,0.192857,False,2005-11-01,11,2005
3,3,3,nytimes,0,-1,2005-11-01 00:00:00,1,0.50000,0.025000,0.250000,False,2005-11-01,11,2005
4,4,4,nytimes,0,-1,2005-11-01 00:00:00,1,-0.03125,-0.050000,0.291667,False,2005-11-01,11,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232098,328086,353781,spiegel,151,-1,2008-03-29 11:47:52,7,0.00000,1.000000,0.000000,False,2008-03-29,3,2008
232099,328087,353782,spiegel,151,-1,2008-03-29 11:47:52,12,0.17965,0.150000,0.000000,False,2008-03-29,3,2008
232100,328088,353783,spiegel,151,-1,2008-03-29 11:47:52,12,0.00000,0.000000,0.000000,False,2008-03-29,3,2008
232101,328089,353784,spiegel,151,-1,2008-03-29 11:47:52,10,0.00000,-0.700000,0.000000,False,2008-03-29,3,2008
